In [1]:
import openai
import json
with open("api_key.json") as f:
    openai.api_key = json.loads(f.read())["key"]

In [43]:
bullets =  {
    "type": "array",
    "items": {
        "type": "object",
        "properties": {
            "main_bullet": {
                "type": "string",
                "description": "a main-bullet for the section being covered"
            },
            "sub_bullets": {
                "type": "array",
                "items": {
                    "type": "string",
                    "description": "sub-bullets for a main-bullet of the section being covered, if applicable"
                }
            }
        },
        "required": ['main_bullet']
    },
    "description": "Bullet points to summarize the section being covered."
}

In [44]:
functions_bp = [{
    "name": "generate_bullet_points",
    "description": "use this function to create bullet points for each section of the primer.",
    "parameters": {
        "type": "object",
        "properties": {
            "sections": {
                "type": "array",
                "items": {
                    "type": "object",
                    "properties": {
                        "bullets": bullets,
                        "title": {
                            "type": "string",
                            "enum": []
                        },
                    }
                },
                "description": "An array of bullet points to return for each section of the primer."
            }
        },
        "required": ["sections"]
    }
}]
functions_n = [{
    "name": "generate_narrations",
    "decscription": "use this function to create the narrations for each section of the primer.",
    "parameters": {
        "type": "object",
        "properties": {
            "sections": {
                "type": "array",
                "items": {
                    "type": "object",
                    "properties": {
                        "title": {
                            "type": "string",
                            "description": "The title of the section being covered."
                        },
                        "narration": {
                            "type": "string",
                            "description": "A narration for a section of the primer."
                        }
                    }
                },
                "description": "An array of narrations for each section of the primer."
            }
        },
        "required": ["sections"]
    },
}]

In [45]:
def set_bullet_title_enums(section_titles):
    functions_bp[0]['parameters']['properties']['sections']['items']['properties']['title']['enum'] = section_titles

In [48]:
def generate_primer(topic, messages=None):
    messages = [] if messages is None else messages
    messages.append({
        "role": "user",
        "content": f"Generate a primer on {topic}. The primer should include an introduction, named sections, and a conclusion. Do not number any of the sections."
    })
    response = openai.ChatCompletion.create(
        model="gpt-3.5-turbo-0613",
        messages=messages, 
    )
    
    messages.append(response["choices"][0]["message"])
    messages.append({"role": "user", "content": "Narrate each section of the primer, as if it were being presented to an audience. Elaborate on the content within each section."})
    response = openai.ChatCompletion.create(
        model="gpt-3.5-turbo-0613",
        messages=messages,
        functions=functions_n,
        function_call="auto",  # auto is default, but we'll be explicit
    )
    narrations = json.loads(response["choices"][0]["message"]['function_call']['arguments'])
    set_bullet_title_enums([section['title'] for section in narrations['sections']])
    
    messages.pop()
    messages.append({"role": "user", "content": "Create bullet points for each section of the primer (including the introduction and conclusion), similar to a PowerPoint presentation."})
    response = openai.ChatCompletion.create(
        model="gpt-3.5-turbo-0613",
        messages=messages,
        functions=functions_bp,
        function_call="auto",  # auto is default, but we'll be explicit
    )
    bullets = json.loads(response["choices"][0]["message"]['function_call']['arguments'])
    set_bullet_title_enums([])
    return messages, bullets, narrations
    


In [49]:
messages, bullets, narrations = generate_primer('Investment Banking')

In [52]:
bullets

{'sections': [{'title': 'Introduction',
   'bullets': [{'main_bullet': 'Investment banking overview',
     'sub_bullets': []}]},
  {'title': 'Definition and Scope',
   'bullets': [{'main_bullet': 'Broad range of financial services',
     'sub_bullets': ['Capital raising',
      'Mergers and acquisitions',
      'Asset management',
      'Sales and trading']}]},
  {'title': 'Investment Banking Services',
   'bullets': [{'main_bullet': 'Capital Raising', 'sub_bullets': []},
    {'main_bullet': 'Mergers and Acquisitions (M&A)', 'sub_bullets': []},
    {'main_bullet': 'Asset Management', 'sub_bullets': []},
    {'main_bullet': 'Sales and Trading', 'sub_bullets': []}]},
  {'title': 'Investment Banking Roles and Departments',
   'bullets': [{'main_bullet': 'Front Office',
     'sub_bullets': ['Corporate finance',
      'Mergers and acquisitions',
      'Sales and trading']},
    {'main_bullet': 'Middle Office',
     'sub_bullets': ['Risk management', 'Compliance', 'Operations']},
    {'main_

In [6]:
from pptx import Presentation
from pptx.util import Inches
import json
import datetime as dt

In [22]:
today = dt.datetime.today().strftime("%m/%d/%Y")

In [110]:
prs = Presentation('BaseTemplate.pptx')

In [4]:
topic = 'Investment Banking'

In [112]:
def set_title_slide(prs, topic, date):
    prs.slides[0].placeholders[0].text = f'Primer on {topic}'
    prs.slides[0].placeholders[1].text = f'Content and Narrations Generated by Chat-GPT \n {date}'

In [113]:
def delete_paragraph(paragraph):
    p = paragraph._p
    parent_element = p.getparent()
    parent_element.remove(p)

def add_bullet(text_frame, bullet, level):
    p = text_frame.add_paragraph()
    run = p.add_run()
    p.level = level
    run.text = bullet

def add_content_to_slide(slide, slide_title, bullets):
    title_frame = slide.placeholders[0].text_frame
    title_frame.text = slide_title
    body_frame = slide.placeholders[1].text_frame
    delete_paragraph(body_frame.paragraphs[0])
    for bullet in bullets:
        main_bullet = bullet['main_bullet']
        add_bullet(body_frame, main_bullet, 0)
        for sub_bullet in bullet.get('sub_bullets', []):
            add_bullet(body_frame, sub_bullet, 1)
        add_bullet(body_frame, "", 0)

In [114]:
def make_presentation(topic, slides_content, template='BaseTemplate.pptx'):
    prs = Presentation(template)
    slide_positions = {}
    set_title_slide(prs, topic, today)
    for i, slide_content in enumerate(slides_content, 2):
        slide = prs.slides.add_slide(prs.slide_layouts[1])
        add_content_to_slide(slide, slide_content['title'], slide_content['bullets'])
        slide_positions[slide_content['title']] = i
    return prs, slide_positions

In [115]:
prs, slide_positions = make_presentation('Investment Banking', bullets['sections'])

In [116]:
prs.save("example.pptx")

In [ ]:
def create_ppt(topic, bullets, outdir='.'):
    prs = Prs()

In [117]:
with open("ib.json", 'w') as f:
    f.write(json.dumps({
        "bullets": bullets,
        "narrations": narrations
    }))

In [7]:
with open('ib.json', 'r') as f:
    data = json.loads(f.read())

In [1]:
import os

In [2]:
def make_dirs(topic_dir):
    os.mkdir(topic_dir)
    os.mkdir(os.path.join(topic_dir, 'Slides'))
    os.mkdir(os.path.join(topic_dir, 'Images'))
    os.mkdir(os.path.join(topic_dir, 'Audio'))
    os.mkdir(os.path.join(topic_dir, 'Configs'))

In [8]:
base_dir = 'Outputs'
topic_dir = os.path.join(base_dir, topic)
if not os.path.isdir(topic_dir):
    make_dirs(topic_dir)

In [13]:
from gtts import gTTS

In [15]:
for narration_info in data['narrations']['sections']:
    tts = gTTS(narration_info['narration'])
    title = narration_info['title']
    audio_dir = os.path.join(topic_dir, 'Audio')
    file_path = os.path.join(audio_dir, f'{title}.mp3')
    tts.save(file_path)

In [11]:
import pyttsx3
converter = pyttsx3.init()

In [12]:
voices = converter.getProperty('voices')
for voice in voices:
    # to get the info. about various voices in our PC 
    print("Voice:")
    print("ID: %s" %voice.id)
    print("Name: %s" %voice.name)
    print("Age: %s" %voice.age)
    print("Gender: %s" %voice.gender)
    print("Languages Known: %s" %voice.languages)

Voice:
ID: HKEY_LOCAL_MACHINE\SOFTWARE\Microsoft\Speech\Voices\Tokens\TTS_MS_EN-US_DAVID_11.0
Name: Microsoft David Desktop - English (United States)
Age: None
Gender: None
Languages Known: []
Voice:
ID: HKEY_LOCAL_MACHINE\SOFTWARE\Microsoft\Speech\Voices\Tokens\TTS_MS_EN-US_ZIRA_11.0
Name: Microsoft Zira Desktop - English (United States)
Age: None
Gender: None
Languages Known: []


In [15]:
for narration_info in data['narrations']['sections']:
    title = narration_info['title']
    audio_dir = os.path.join(topic_dir, 'Audio')
    file_path = os.path.join(audio_dir, f'{title}.mp3')
    converter.save_to_file(narration_info['narration'], file_path)
    converter.runAndWait()